In [ ]:
def install_energyplus(SHARED_DIR, MODELS_FOLDER):

    import os
    import shutil
    import tarfile

    # Copy eneryplus model to personal model folder
    tar_filename = 'EnergyPlus-25.1.0-68a4a7c774-Linux-Ubuntu22.04-x86_64.tar.gz'
    SHARED_TAR_FILE = os.path.join(SHARED_DIR, tar_filename)
    print(f'> SHARED TAR FILE: ', SHARED_TAR_FILE)
    PERSONAL_TAR_FILE = os.path.join(MODELS_FOLDER, tar_filename)
    print(f'> PERSONAL TAR FILE: ', PERSONAL_TAR_FILE)

    # The full path to the EnergyPlus tar.gz file.
    tar_filepath = os.path.join(MODELS_FOLDER,tar_filename)

    if not os.path.exists(tar_filepath):
      print(f"Copying '{tar_filename}' into '{MODELS_FOLDER}'...")
      try:
        shutil.copy(SHARED_TAR_FILE, PERSONAL_TAR_FILE)
        print("File copied successfully.")
      except Exception as e:
        print(f"Error copying file: {e}")
    else:
      print(f"File '{tar_filename}' already exists in '{MODELS_FOLDER}'.")

    # The name of the directory that will be created after extraction.
    # This is typically the name of the archive without the .tar.gz extension.
    extracted_folder_name = tar_filename.replace('.tar.gz','')
    # extracted_folder_name = 'EnergyPlus'

    # The final path where EnergyPlus will be installed.
    install_dir = os.path.join(MODELS_FOLDER, extracted_folder_name)

    # Handle Existing Installation
    # Check if the installation directory already exists.
    # If the directory exists it will remove it to ensure a clean installation
    print(f"\nChecking for existing installation at: {install_dir}")
    if os.path.exists(install_dir):
        print("Existing EnergyPlus folder found. Removing it to ensure a fresh installation...")
        try:
            shutil.rmtree(install_dir)
            print("Existing folder removed successfully.")
        except Exception as e:
            print(f"\nERROR: Could not remove existing directory: {e}")
            raise
    else:
        print("No existing installation found. Proceeding.")

    # Check for Installation file
    # Extract the archive
    # --- 4. Check for Installation File ---
    print(f"\nChecking for installation file at: {tar_filepath}")
    if not os.path.exists(tar_filepath):
        print("\nERROR: Installation file not found!")
        print(f"Please make sure that '{tar_filename}' exists in the directory: '{MODELS_FOLDER}' in your Google Drive.")
        # Stop execution if the file doesn't exist.
        raise FileNotFoundError(f"The file {tar_filepath} was not found.")
    else:
        print("Installation file found.")

    # --- 5. Extract the Archive ---
    print(f"\nExtracting '{tar_filename}' to '{MODELS_FOLDER}'...")
    try:
        with tarfile.open(tar_filepath, 'r:gz') as tar:
            tar.extractall(path=MODELS_FOLDER)
        print("Extraction complete.")
        print(f"EnergyPlus has been extracted to: {install_dir}")
    except Exception as e:
        print(f"\nERROR: An error occurred during extraction: {e}")
        raise

    # Add EnergyPlus to System PATH and verify installation
    print("\nAdding EnergyPlus to the system PATH...")
    eplus_bin_path = install_dir

    if eplus_bin_path not in os.environ['PATH']:
        os.environ['PATH'] += f":{eplus_bin_path}"
        print("PATH updated successfully.")
    else:
        print("PATH already includes the EnergyPlus directory.")

    print("\nVerifying EnergyPlus installation...")

    verification_status = os.system('energyplus --version')

    if verification_status == 0:
        print("\n-------------------------------------------")
        print("✅ EnergyPlus installation successful!")
        print("-------------------------------------------")
        print("You can now run EnergyPlus simulations in this Colab session.")
    else:
        print("\n-------------------------------------------------")
        print("❌ Verification failed. Something went wrong.")
        print("-------------------------------------------------")
        print("Please check the output above for errors.")




In [ ]:
def run_energyplus(MODEL_INPUT_FOLDER, MODEL_OUTPUT_FOLDER):

    import os
    import glob
    import subprocess

    # Get the scenario_name from the OUTPUT_FOLDER
    scenario_name = os.path.basename(MODEL_OUTPUT_FOLDER)

    # --- Define paths ---
    idf_dir = os.path.join(MODEL_INPUT_FOLDER, 'IDF')
    epw_dir = os.path.join(MODEL_INPUT_FOLDER, 'EPW')

    print(f'Running EnergyPlus for: \n --> {scenario_name} scenario \nMake sure this is the scenario you want to run EnergyPlus for')

    # --- Find .epw file ---
    print("\nSearching for weather file in:", epw_dir)
    epw_files = glob.glob(os.path.join(epw_dir, '*.epw'))
    if not epw_files:
        raise FileNotFoundError("No .epw weather files found: copy file from base scenario and edit what you want.")
    epw_filepath = epw_files[0]
    print(f"✅ Automatically selected weather file: {os.path.basename(epw_filepath)}")

    # --- Find .idf files ---
    print("\nVerifying IDF path...")
    os.makedirs(MODEL_OUTPUT_FOLDER, exist_ok=True)
    idf_files = glob.glob(os.path.join(idf_dir, '*.idf'))
    if not idf_files:
        raise FileNotFoundError("No IDF files found: copy file from base scenario and edit what you want..")
    print(f"Found {len(idf_files)} IDF file(s) to simulate.")

    # --- Run simulations ---
    for idf_path in idf_files:
        idf_basename = os.path.splitext(os.path.basename(idf_path))[0]
        print(f"\n{'='*60}")
        print(f"Starting simulation for: {idf_basename}.idf")
        print(f"{'='*60}")

        output_subdir = os.path.join(MODEL_OUTPUT_FOLDER)
        os.makedirs(output_subdir, exist_ok=True)
        print(f"Results will be saved in: {output_subdir}")

        print("\nRunning EnergyPlus...")
        eplus_command = [
            'energyplus',
            '--weather', epw_filepath,
            '--output-directory', output_subdir,
            idf_path
        ]

        eplus_process = subprocess.run(eplus_command, capture_output=True, text=True)

        if eplus_process.returncode == 0:
            print("✅ EnergyPlus simulation completed successfully.")
        else:
            print("❌ ERROR: EnergyPlus simulation failed.")
            print("--- EnergyPlus STDOUT ---")
            print(eplus_process.stdout)
            print("--- EnergyPlus STDERR ---")
            print(eplus_process.stderr)

    print(f"\n{'='*60}")
    print("All simulations have been processed.")
    print(f"{'='*60}")


In [ ]:
def add_comfort_assessment_to_csv(city, MODEL_INPUT_FOLDER, MODEL_OUTPUT_FOLDER):

    import pandas as pd
    import os
    import glob

    epw_dir = os.path.join(MODEL_INPUT_FOLDER, 'EPW')
    eplusout_path = os.path.join(MODEL_OUTPUT_FOLDER, 'eplusout.csv')
    analysis_dir = os.path.join(MODEL_OUTPUT_FOLDER, 'analysis')
    os.makedirs(analysis_dir, exist_ok=True)
    print(f"Results will be saved in: {analysis_dir}")


    # Find all eplusout.csv files in subdirectories of Results
    eplusout_files = glob.glob(os.path.join(MODEL_OUTPUT_FOLDER, 'eplusout.csv'))

    # Process each eplusout.csv file
    for eplusout_path in eplusout_files:
        print(f"\nProcessing: {os.path.basename(eplusout_path)}")
        folder_name = os.path.basename(os.path.dirname(eplusout_path))

        # Load eplusout.csv
        eplusout = pd.read_csv(eplusout_path)

        # Find the column ending with ":Zone Operative Temperature C"
        operative_cols = [col for col in eplusout.columns if col.endswith(":Zone Operative Temperature [C](Hourly)")]
        if not operative_cols:
            print(f"No operative temperature column found in {eplusout_path}")
            continue
        operative_col = operative_cols[0]

        # Load the PrevailingAndNeutral_*.csv file
        prevailing_files = glob.glob(os.path.join(epw_dir, 'PrevailingAndNeutral_*.csv'))
        if not prevailing_files:
            print("No PrevailingAndNeutral_*.csv file found.")
            continue
        prevailing_path = prevailing_files[0]
        PrevailingAndNeutral = pd.read_csv(prevailing_path)

        # Create new dataframe
        new_df = PrevailingAndNeutral.copy()
        new_df["Operative Temperature"] = eplusout[operative_col].values

        # Add EN-16798 Cat II Upper limit
        new_df["EN-16798_Cat II_Limit"] = new_df["NeutralTemp"] + 3.5

        # Add Yes_No_Overheating column
        new_df["Yes_No_Overheating"] = (new_df["Operative Temperature"] > new_df["EN-16798_Cat II_Limit"]).astype(int)

        # Add Distance_from_comfort column
        new_df["Distance_from_comfort"] = new_df.apply(
            lambda row: 0 if row["Operative Temperature"] <= row["EN-16798_Cat II_Limit"]
            else row["Operative Temperature"] - row["EN-16798_Cat II_Limit"], axis=1)

        # Add new columns for temperature thresholds
        # ENSURE THESE ARE THE EXACT COLUMN NAMES FROM YOUR PrevailingAndNeutral.csv FILE

        if city == 'Davao':
          new_df["Hours_1°C_From_Comfort"] = (new_df["Operative Temperature"] > new_df["Plus_4.5"]).astype(int)
          new_df["Hours_2°C_From_Comfort"] = (new_df["Operative Temperature"] > new_df["Plus_5.5"]).astype(int)
          new_df["Hours_3°C_From_Comfort"] = (new_df["Operative Temperature"] > new_df["Plus_6.5"]).astype(int)
          new_df["Hours_4°C_From_Comfort"] = (new_df["Operative Temperature"] > new_df["Plus_7.5"]).astype(int)
        elif city == 'Timbuktu':
          new_df["Hours_1°C_From_Comfort"] = (new_df["Operative Temperature"] > new_df["Plus_4.5-SlightlyWarm"]).astype(int)
          new_df["Hours_2°C_From_Comfort"] = (new_df["Operative Temperature"] > new_df["Plus_5.5-SlightlyWarm_2"]).astype(int)
          new_df["Hours_3°C_From_Comfort"] = (new_df["Operative Temperature"] > new_df["Plus_6.5-Hot"]).astype(int)
          new_df["Hours_4°C_From_Comfort"] = (new_df["Operative Temperature"] > new_df["Plus_7.5-VeryHot"]).astype(int)

        # Save the new dataframe
        output_filename = f"Comfort_analysis_{folder_name}.csv"
        output_path = os.path.join(analysis_dir, output_filename)
        new_df.to_csv(output_path, index=False)
        print(f"Saved: {output_path}")


In [ ]:
def add_drybulb_to_comfort(MODEL_INPUT_FOLDER, MODEL_OUTPUT_FOLDER):

    import os
    import glob
    import pandas as pd

    # Set the paths
    epw_dir = os.path.join(MODEL_INPUT_FOLDER, 'EPW')
    analysis_dir = os.path.join(MODEL_OUTPUT_FOLDER, 'analysis')

    # Load all .epw files as dataframes, skipping the first 8 lines
    epw_dataframes = []
    epw_files = glob.glob(os.path.join(epw_dir, '*.epw'))
    for epw_file in epw_files:
        try:
            df = pd.read_csv(epw_file, skiprows=8, header=None)
            epw_dataframes.append(df.iloc[:, 6].reset_index(drop=True))  # 7th column (index 6)
        except Exception as e:
            print(f"Error reading {epw_file}: {e}")

    # Find all Comfort_analysis_*.csv files in the analysis folder
    comfort_files = glob.glob(os.path.join(analysis_dir, 'Comfort_analysis_*.csv'))

    # Append outdoor_temperature column to each comfort analysis file
    for comfort_file in comfort_files:
        try:
            comfort_df = pd.read_csv(comfort_file)
            # Use the first available outdoor temperature series
            if epw_dataframes:
                outdoor_temp = epw_dataframes[0][:len(comfort_df)].reset_index(drop=True)
                comfort_df['outdoor_temperature'] = outdoor_temp
                comfort_df.to_csv(comfort_file, index=False)
                print(f"Updated: {comfort_file}")
            else:
                print("No EPW data available to append.")
        except Exception as e:
            print(f"Error updating {comfort_file}: {e}")


In [ ]:
def plot_heatmap(MODEL_OUTPUT_FOLDER, figname):

    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.colors import ListedColormap, BoundaryNorm

    scenario_name = os.path.basename(MODEL_OUTPUT_FOLDER)
    analysis_dir = os.path.join(MODEL_OUTPUT_FOLDER, 'analysis')


    # Helper function to clean and parse Date/Time
    def parse_datetime_column(datetime_series):
        cleaned = datetime_series.astype(str).str.strip().str.replace(r'\s+', ' ', regex=True)
        parsed = pd.to_datetime(cleaned, format='%m/%d %H:%M:%S', errors='coerce')
        return parsed

    # Reshape data into 2D matrix (hours x days)
    def reshape_to_heatmap(data_series, datetime_index):
        df = pd.DataFrame({'value': data_series.values}, index=datetime_index)
        df = df.dropna()
        df['day'] = df.index.date
        df['hour'] = df.index.hour
        heatmap_data = df.pivot_table(index='hour', columns='day', values='value')
        heatmap_data = heatmap_data.sort_index(ascending=False)  # Reverse hours to start from 23 at bottom
        return heatmap_data

    # Process each CSV file
    for file_name in os.listdir(analysis_dir):
        if not file_name.endswith('.csv'):
            continue

        print(f"\n=== {file_name} ===\n")
        df = pd.read_csv(os.path.join(analysis_dir, file_name))

        # Parse Date/Time
        df['Parsed_Date'] = parse_datetime_column(df['Date/Time'])
        df.set_index('Parsed_Date', inplace=True)

        # Prepare plots
        fig, axes = plt.subplots(4, 1, figsize=(18, 18), sharey=True)
        fig.suptitle(f'{scenario_name} scenario', fontsize=20, fontweight='bold')
        # Plot 1: Outdoor Temperature (inverted gradient: blue to red)
        temp_data = reshape_to_heatmap(df['outdoor_temperature'], df.index)
        im1 = axes[0].imshow(temp_data, aspect='auto', cmap='RdYlBu_r', origin='lower')
        axes[0].set_title("Outdoor dry bulb temperature [°C]")
        fig.colorbar(im1, ax=axes[0], orientation='vertical')

        # Plot 2: Operative Temperature
        operative_data = reshape_to_heatmap(df['Operative Temperature'], df.index)
        im2 = axes[1].imshow(operative_data, aspect='auto', cmap='RdYlBu_r', origin='lower')
        axes[1].set_title("Indoor dry bulb temperature [°C]")
        fig.colorbar(im2, ax=axes[1], orientation='vertical')

        # Plot 3: Overheating Hours
        overheat_data = reshape_to_heatmap(df['Yes_No_Overheating'], df.index)
        cmap3 = ListedColormap(['forestgreen', 'darkred'])
        bounds = [0, 0.5, 1]
        norm = BoundaryNorm(bounds, cmap3.N)
        im3 = axes[2].imshow(overheat_data, aspect='auto', cmap=cmap3, norm=norm, origin='lower')
        axes[2].set_title("Overheating hours")
        cbar3 = fig.colorbar(im3, ax=axes[2], orientation='vertical', ticks=[0, 1])
        cbar3.ax.set_yticklabels(['0', '1'])

        # Plot 4: Distance from Comfort
        comfort_data = reshape_to_heatmap(df['Distance_from_comfort'], df.index)
        im4 = axes[3].imshow(comfort_data, aspect='auto', cmap='YlOrRd', origin='lower')
        axes[3].set_title("Distance from comfort")
        fig.colorbar(im4, ax=axes[3], orientation='vertical')

        # Format axes
        for ax in axes:
            ax.set_xlabel("Day")
            ax.set_ylabel("Hour of Day")
            ax.set_xticks(np.arange(0, len(temp_data.columns), 30))
            ax.set_xticklabels([pd.to_datetime(d).strftime('%m/%d') for i, d in enumerate(temp_data.columns) if i % 30 == 0])
            ax.set_yticks(np.arange(0, 24))
            ax.set_yticklabels(list(reversed(range(24))))

        plt.tight_layout(rect=[0, 0, 1, 0.96])

        # Save the figure as a .png image at 300 DPI
        plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
        print("File saved:", os.path.exists(figname))

        plt.show()
        plt.close(fig)

In [ ]:
def plot_overheating_assessment(MODEL_OUTPUT_FOLDER, figname, scenarios):

    import pandas as pd
    import os
    import glob
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import numpy as np

    # Process each scenario
    for scenario in scenarios:
        try:

            file_path = os.path.join(
                os.path.dirname(MODEL_OUTPUT_FOLDER.rstrip('/')),
                scenario,
                'analysis',
                f"Comfort_analysis_{scenario}.csv"
            )
            df = pd.read_csv(file_path)

            # Check if required columns exist
            required_columns = [
                "Yes_No_Overheating",
                "Hours_1°C_From_Comfort",
                "Hours_2°C_From_Comfort",
                "Hours_3°C_From_Comfort",
                "Hours_4°C_From_Comfort"
            ]
            if not all(col in df.columns for col in required_columns):
                print(f"Skipping {file_path} due to missing required columns.")
                continue

            # Create summary values
            summary_values = [
                df["Yes_No_Overheating"].sum(),
                df["Hours_1°C_From_Comfort"].sum(),
                df["Hours_2°C_From_Comfort"].sum(),
                df["Hours_3°C_From_Comfort"].sum(),
                df["Hours_4°C_From_Comfort"].sum()
            ]

            summary_labels = [
                "hours above\ncomfort",
                "Total hours\n1°C above Comfort",
                "Total hours\n2°C above Comfort",
                "Total hours\n3°C above Comfort",
                "Total hours\n4°C above Comfort"
            ]

            # Print summary table
            summary_df = pd.DataFrame({
                "Metric": summary_labels,
                "Total": summary_values
            })
            print(f"\nSummary for file: {os.path.basename(file_path)}")
            print(summary_df)

            # Generate inverted Reds color gradient
            norm = plt.Normalize(min(summary_values), max(summary_values))
            colors = cm.Reds_r(norm(summary_values))

            # Plot bar chart using the summary values
            fig = plt.figure(figsize=(5, 3))  # Reduced size
            bars = plt.bar(summary_labels, summary_values, color=colors, edgecolor='black')
            plt.title(f"Comfort analysis {scenario}")
            plt.ylabel("Total Hours")
            plt.xticks(rotation=90, ha='center')  # Vertical labels

            # Add more Y-axis ticks
            max_val = max(summary_values)
            plt.yticks(np.linspace(0, max_val, num=10))

            plt.tight_layout()
            plt.grid(axis='y', linestyle='--', alpha=0.7)

            # Save the figure as a .png image at 300 DPI
            figname = figname.replace('.png', f'_{scenario}.png')
            plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
            print("File saved:", os.path.exists(figname))

            plt.show()

        except Exception as e:
            print(f"Error processing {file_path}: {e}")


In [ ]:
def plot_hottest_week_of_the_year(MODEL_OUTPUT_FOLDER, figname, scenarios):

    import pandas as pd
    import matplotlib.pyplot as plt
    import os
    import glob

    # Define the row range for the hottest week:
    if 'Timbuktu' in MODEL_OUTPUT_FOLDER:
      start_row = 3336  # 20/05/2013
      end_row = 3504  # 26/05/2013
    elif 'Davao' in MODEL_OUTPUT_FOLDER:
      start_row = 2712  # 24/04/2018
      end_row = 2880  # 30/04/2018


    # Initialize the plot
    plt.figure(figsize=(10, 5))

    # Initialize x_ticks and x_tick_labels
    x_ticks = []
    x_tick_labels = []


    # Process each scenario
    for idx,scenario in enumerate(scenarios):

        file_path = os.path.join(
            os.path.dirname(MODEL_OUTPUT_FOLDER.rstrip('/')),
            scenario,
            'analysis',
            f"Comfort_analysis_{scenario}.csv"
        )
        df = pd.read_csv(file_path)

        # Extract the file name without prefix and extension
        file_name = os.path.basename(file_path).replace("Comfort_analysis_", "").replace(".csv", "")

        # Slice the data for the hottest week
        df_week = df.iloc[start_row:end_row]

        # Plot outdoor temperature from the first file as dashed black line
        if idx == 0 and "outdoor_temperature" in df_week.columns:
            plt.plot(df_week["outdoor_temperature"].values, color='black', linestyle='--', label='Outdoor temperature')

        # Plot operative temperature
        if "Operative Temperature" in df_week.columns:
            plt.plot(df_week["Operative Temperature"].values, label=f"Indoor temperature {file_name}")

        # Store x-axis labels from the first file
        if idx == 0 and "Date/Time" in df_week.columns:
            x_labels = df_week["Date/Time"].astype(str).values
            x_ticks = list(range(0, len(x_labels), 24))
            x_tick_labels = [x_labels[i][:6] for i in x_ticks]

    # Set plot titles and labels
    plt.title("Temperature Profile During Heatwave")
    plt.xlabel("Heatwave days")
    plt.ylabel("Temperature [°C]")

    # Set x-axis ticks and labels
    if x_ticks and x_tick_labels:
        plt.xticks(ticks=x_ticks, labels=x_tick_labels, rotation=45, ha='right')

    # Move legend below the plot
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), ncol=2)
    plt.grid(True)
    plt.tight_layout()

    # Save the figure as a .png image at 300 DPI
    figname = figname.replace('.png', f'_{scenario}.png')
    plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
    print("File saved:", os.path.exists(figname))

    plt.show()